In [1]:
import sys, os
sys.path.insert(0, '/home/hektor/scripts')
#sys.path.insert(0, '/lunarc/nobackup/projects/qim')

import qim.registration as registration
import qim.tools as tools
import tifffile
import numpy as np
import matplotlib.pyplot as plt
%matplotlib widget

from skimage import filters
import scipy.optimize as opt


## Fabricate some data

In [2]:
xedges = np.linspace(0,64,64)
yedges = xedges
xcen = tools.find_bin_centers(xedges)
ycen = tools.find_bin_centers(yedges)
Y,X = np.meshgrid(ycen,xcen,indexing='ij') #coordinates in dual histogram
xy = np.vstack((X.ravel(),Y.ravel())) #to comply with curve_fit syntax

data = tools.gaussian2D(xy,10,12,20,10,np.pi/6,10,0)+tools.gaussian2D(xy,40,30,5,5,np.pi/8,5,0)+tools.gaussian2D(xy,20,50,20,5,np.pi/8,15,0)
data += tools.gaussian2D(xy,45,10,2,4,np.pi/2,10,0)
data += tools.gaussian2D(xy,15,50,2,4,np.pi/2,10,0)

data=data.reshape(X.shape)
#data /= data.sum()
print(X.shape,data.shape)
plt.figure()
plt.pcolormesh(X,Y,data)

(63, 63) (63, 63)


Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [2]:
qimfolder = '/lunarc/nobackup/projects/qim/QIM_2020_LU_Elin_Bio'
datapath = os.path.join(qimfolder,'pvt21/reg-common-peaks_new')
x_data = os.path.join(datapath,'x_registered.tiff')
n_data = os.path.join(datapath,'n-bin1.tif')

x_volume = tifffile.imread(x_data)
n_volume = tifffile.imread(n_data)


In [3]:
nbins = 128
data,xedges,yedges = registration.make_dual_histogram(x_volume,n_volume,nbins)

registration.plot_dual_histogram(data,xedges,yedges,xlabel='X-ray intensity',ylabel='Neutron intensity',vmax=2500)

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [8]:
registration.plot_dual_histogram(np.log(data),xedges,yedges,xlabel='X-ray intensity',ylabel='Neutron intensity')

/sw/easybuild/software/IPython/7.9.0-foss-2019b-Python-3.7.4/lib/python3.7/site-packages/ipykernel_launcher.py:1: RuntimeWarning: divide by zero encountered in log
  """Entry point for launching an IPython kernel.


Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [23]:
#should go in dualHistogramSegmentation.py
def fit_gaussians(H,ml,xedges,yedges,fitdist=20,mindist=10):
    """ 
    Fit 2D gaussians to the dual histogram.
    Input:
        H: dual histogram
        ml: marker locations
        xedges: centers of bins on x-axis
        yedges: centers of bins on y-axis
    Keyword arguments:
        fitdist: Radius of circle to fit data within, default = 20
        mindist: Smallest allowed distance between two Gaussians, default = 10
    Output:
        opts: list of fitted parameters [xc,yc,sigma_x,sigma_y,theta,amplitude,offset]
    """
    xcen = tools.find_bin_centers(xedges)
    ycen = tools.find_bin_centers(yedges)

    Y,X = np.meshgrid(ycen,xcen,indexing='ij') #coordinates in dual histogram
    xy = np.vstack((X.ravel(),Y.ravel())) #to comply with curve_fit syntax
    opts = []
    
    #Sort the markers to always fit the highest first
    hv = [H[m[1],m[0]] for m in ml]
    idx = np.argsort(hv)
    ml = [ml[i] for i in idx]
    
    for m in reversed(ml):
        x0,y0 =xcen[m[0]],ycen[m[1]] 
        print('Trying to fit peak at ({:.1f}, {:.1f})'.format(x0,y0))
        Hc=H.copy()
        #subtract already fitted
        if len(opts)>0:
            for op in opts:
                g = tools.gaussian2D(xy,*op)
                Hc -= g.reshape(Hc.shape)
        for i in range(Hc.shape[0]):
            for j in range(Hc.shape[1]):
                d = np.sqrt((i-m[1])**2+(j-m[0])**2)
                if d > fitdist:
                    Hc[i,j] = 0
        #check if marker is on an edge
        if (m[0]==0) or (m[0]==H.shape[0]-1) or (m[1]==0) or (m[1]==H.shape[0]-1):
            print('Marker is on an edge')
            Hc = np.zeros(H.shape)
            if (m[0]==0) or (m[0]==H.shape[0]-1): #left/right edges
                Hc[:,m[0]] = H[:,m[0]]
            elif (m[1]==0) or (m[1]==H.shape[0]-1): #bottom/top edges
                Hc[m[1],:] = H[m[1],:]
                
        # Find the amplitude and position of highest value
        idx = np.argmax(Hc)
        y0i,x0i = np.unravel_index(idx,Hc.shape)
        x0 = X[y0i,x0i]
        y0 = Y[y0i,x0i]
        H0=Hc[y0i,x0i]
        of = 0 #offset
        inits = [100,100,0]
       
        try:
            pt,pcov = opt.curve_fit(lambda xy,sx,sy,th: tools.gaussian2D(xy,x0,y0,sx,sy,th,H0,of),xy,Hc.ravel(),p0=inits)
            popt=[x0,y0,pt[0],pt[1],pt[2],H0,of]
            print('Fitted parameters [{:.3f} {:.3f} {:.3f} {:.1f} {:.1f} {:1f} {:.1f}]'.format(*popt))
            # sanity check
            tr = np.trace(pcov)
            det = np.linalg.det(pcov)
            if (tr<=0) and (det<=0):
                print('This is not fitted to a maximum, determinant: {:.3f}, trace: {:.3f}. Skipping.'.format(tr,det))
                continue
            #Check if it is close to something already fitted
            if len(opts)>0:
                dist = [np.sqrt((popt[0]-op[0])**2+(popt[1]-op[1])**2) for op in opts]
                if np.min(dist)>mindist:
                    opts.append(popt)
                    print('Fitted {:d} peaks'.format(len(opts)))
                else:
                    print('This Gaussian is too close to something already fitted, d={:.3f}. Skipping.'.format(np.min(dist)))
            else:
                opts.append(popt)
                print('Fitted {:d} peaks'.format(len(opts)))
        except RuntimeError:
            print('Failed to fit this peak. Moving to the next.')
            continue
    return opts
    
def get_pdfs(xedges,yedges,opts):
    """
    Compute the probability densities of the fitted Gaussians
    """
    xcen = tools.find_bin_centers(xedges)
    ycen = tools.find_bin_centers(yedges)
    #bin size
    dx = np.diff(xcen)[0] 
    dy = np.diff(xcen)[1] 
    
    Y,X = np.meshgrid(ycen,xcen,indexing='ij') #coordinates in dual histogram
    xy = np.vstack((X.ravel(),Y.ravel())) #to comply with curve_fit syntax
    pdfs = np.zeros((len(opts),*X.shape))
    for i,op in enumerate(opts):
        sx,sy,th = op[2],op[3],op[4]
        a = np.cos(th)*np.cos(th)/(2*sx**2)+np.sin(th)*np.sin(th)/(2*sy**2)
        b = -np.sin(2*th)/(4*sx**2)+np.sin(2*th)/(4*sy**2)
        c = np.sin(th)*np.sin(th)/(2*sx**2)+np.cos(th)*np.cos(th)/(2*sy**2)
        Si = np.array([[a,b],[b,c]]) #Inverse covariance matrix
        S = np.linalg.inv(Si)
        detS = np.linalg.det(S)
        mu = np.array([op[0],op[1]])
        pdf = np.zeros(xy.shape[1])
        for k,phi in enumerate(xy.T):
            pdf[k] = 1./(2*np.pi*np.sqrt(detS))*np.exp(-0.5*(np.dot(phi-mu,np.dot(Si,phi-mu))))*dx*dy
        pdfs[i]=pdf.reshape(X.shape)
    return pdfs


In [3]:
def fit_gaussians(H,ml,xedges,yedges,fitdist=20,mindist=10):
    """ 
    Fit 2D gaussians to the dual histogram.
    Input:
        H: dual histogram
        ml: marker locations
        xedges: centers of bins on x-axis
        yedges: centers of bins on y-axis
    Keyword arguments:
        fitdist: Radius of circle to fit data within. Scalar or list. default = 20
        mindist: Smallest allowed distance between two Gaussians, default = 10
    Output:
        opts: list of fitted parameters [xc,yc,sigma_x,sigma_y,theta,amplitude,offset]
    """
    xcen = tools.find_bin_centers(xedges)
    ycen = tools.find_bin_centers(yedges)

    Y,X = np.meshgrid(ycen,xcen,indexing='ij') #coordinates in dual histogram
    xy = np.vstack((X.ravel(),Y.ravel())) #to comply with curve_fit syntax
    opts = []
    
    #Sort the markers to always fit the highest first
    hv = [H[m[1],m[0]] for m in ml]
    idx = np.argsort(hv)
    ml = [ml[i] for i in idx]
    try:
        fitdist = [fitdist[i] for i in idx]
        fitdist.reverse()
    except: #fitdist wasn't a list
        pass 
    for i,m in enumerate(reversed(ml)):
        x0,y0 =xcen[m[0]],ycen[m[1]] 
        print('Trying to fit peak at ({:.1f}, {:.1f})'.format(x0,y0))
        try:
            fd = fitdist[i]
        except: #fitdist wasn't a list
            fd = fitdist
        print('Removing everything outside of {:.3f} pixels'.format(fd))
        Hc=H.copy()
        #subtract already fitted
        if len(opts)>0:
            for op in opts:
                g = tools.gaussian2D(xy,*op)
                Hc -= g.reshape(Hc.shape)
        for i in range(Hc.shape[0]):
            for j in range(Hc.shape[1]):
                d = np.sqrt((i-m[1])**2+(j-m[0])**2)
                if d > fd:
                    Hc[i,j] = 0
        #check if marker is on an edge
        if (m[0]==0) or (m[0]==H.shape[0]-1) or (m[1]==0) or (m[1]==H.shape[0]-1):
            print('Marker is on an edge')
            Hc = np.zeros(H.shape)
            if (m[0]==0) or (m[0]==H.shape[0]-1): #left/right edges
                Hc[:,m[0]] = H[:,m[0]]
            elif (m[1]==0) or (m[1]==H.shape[0]-1): #bottom/top edges
                Hc[m[1],:] = H[m[1],:]
                
        # Find the amplitude and position of highest value
        idx = np.argmax(Hc)
        y0i,x0i = np.unravel_index(idx,Hc.shape)
        x0 = X[y0i,x0i]
        y0 = Y[y0i,x0i]
        H0=Hc[y0i,x0i]
        of = 0 #offset
        inits = [100,100,0]
       
        try:
            pt,pcov = opt.curve_fit(lambda xy,sx,sy,th: tools.gaussian2D(xy,x0,y0,sx,sy,th,H0,of),xy,Hc.ravel(),p0=inits)
            popt=[x0,y0,pt[0],pt[1],pt[2],H0,of]
            print('Fitted parameters [{:.3f} {:.3f} {:.3f} {:.1f} {:.1f} {:1f} {:.1f}]'.format(*popt))
            #Check if it is close to something already fitted
            if len(opts)>0:
                dist = [np.sqrt((popt[0]-op[0])**2+(popt[1]-op[1])**2) for op in opts]
                if np.min(dist)>mindist:
                    opts.append(popt)
                    print('Fitted {:d} peaks'.format(len(opts)))
                else:
                    print('This Gaussian is too close to something already fitted, d={:.3f}. Skipping.'.format(np.min(dist)))
            else:
                opts.append(popt)
                print('Fitted {:d} peaks'.format(len(opts)))
        except RuntimeError:
            print('Failed to fit this peak. Moving to the next.')
            continue
    return opts


In [4]:
#ml = [[64,64],[125,200],[200,100],[125,150]]
#ml = [[10,12],[40,30],[45,10],[15,50]]
#ml = [[45,10],[15,50]]
ml = [[20800, 46000],  [48000, 28000], [65085, 16200]]#x reg onto n
markers = registration.setup_markers(data,xedges,yedges,markers=ml)
print(markers)
opts = registration.fit_gaussians(data,markers,xedges,yedges,fitdist=[20,40,40],mindist=20)
registration.plot_dual_histogram(data,xedges,yedges,markers=markers,fits=opts,vmax=2500)

[[41, 90], [94, 55], [127, 32]]
Trying to fit peak at (65279.0, 16639.7)
Removing everything outside of 40.000 pixels
Marker is on an edge
Fitted parameters [65279.004 15103.770 109.559 1503.9 0.0 311183.000000 0.0]
Fitted 1 peaks
Trying to fit peak at (48383.3, 28415.6)
Removing everything outside of 40.000 pixels
Fitted parameters [48895.254 29439.551 5489.366 2411.6 -0.1 73323.000000 0.0]
Fitted 2 peaks
Trying to fit peak at (21247.7, 46335.3)
Removing everything outside of 20.000 pixels
Fitted parameters [17663.730 46335.293 3291.184 5658.4 -0.3 6293.000000 0.0]
Fitted 3 peaks


Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [5]:
pdfs = registration.get_pdfs(xedges,yedges,opts)
for p in pdfs:
    p /=p.max()

#p = np.zeros(data.shape)
#for pd in pdfs:
#    p += pd

plt.figure()
plt.imshow(p,origin='lower')
plt.colorbar()
for p in pdfs:
    print(p.sum())

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

10.456659699366755
624.7665574466951
886.6338280531127


In [24]:
def voxel_coverage(H,xedges,yedges,opts,coverage=0.99):
    """
    Finds the minimum distance between the histogram and the fitted curves such that the given percentage of the histogram is labelled 
    Input:
        H: dual histogram
        xedges: centers of bins on x-axis
        yedges: centers of bins on y-axis
        opts: parameters for fitted curves
    Keyword arguments:
        coverage: minimum coverage of dual histogram, default=0.99
    Output:
        sigma: minimal distance threshold
    """
    xcen = tools.find_bin_centers(xedges)
    ycen = tools.find_bin_centers(yedges)
    phase = np.zeros(H.shape,dtype=int)
    vxlcov = 0
    sigma = 1
    while vxlcov <= coverage:
        sigma += 10**int(np.log10(sigma))
        phase = _phase_diagram(H,xcen,ycen,opts,sigma)

        vxlcov = H[phase>0].sum()/(H.sum())#-H[0,0]) #subtract the background bin
        print("Finding full coverage: sigma={}, coverage={}".format(sigma,vxlcov))
    return sigma

def _phase_diagram(H,xcen,ycen,opts,sigma):
    phase = np.zeros(H.shape,dtype=int)
    for ix,x in enumerate(xcen):
        for iy,y in enumerate(ycen):
            if H[iy,ix]>0:
                distances = [tools.distance(x,y,o) for o in opts]
                i = np.argmin(distances)
                distanceMin = distances[i]
                if distanceMin < sigma: 
                    phase[iy,ix] = i+1
    return phase

def phase_diagram(H,xedges,yedges,opts,sigma=None,coverage=0.99):
    """
    Labels the dual histogram
    Input:
        H: dual histogram
        xedges: edges of bins on x-axis
        yedges: edges of bins on y-axis
        opts: parameters for fitted curves
    Keyword arguments:
        coverage: minimum percentage of the histogram to label, default=0.99
        sigma: max distance to consider, default=None which means it will be computed based on the coverage
    Output:
        phasediagram: labelled dual histogram
    """
    xcen = tools.find_bin_centers(xedges)
    ycen = tools.find_bin_centers(yedges)
    if not sigma:
        sigma = voxel_coverage(H,xcen,ycen,opts,coverage=coverage)
    phasediagram = _phase_diagram(H,xcen,ycen,opts,sigma)
    return phasediagram

In [42]:
coverage = 0.2
phasediagram = phase_diagram(data,xedges,yedges,opts,coverage=coverage,sigma=3)
nphases=phasediagram.max()+1

#filter based on pdf
#phasediagram = np.where(p>0.001,phasediagram,0)

In [36]:
ph = np.argmax(pdfs,axis=0)+1
ph = np.where(np.max(pdfs,axis=0)>1-0.992,ph,0)

In [43]:
#plot the phasediagram
cmap = tools.random_cmap(ncolors=nphases,startmap=plt.cm.Set3)#'Set3' #vill bara ha x färger
Y,X = np.meshgrid(yedges,xedges,indexing='ij') #coordinates in dual histogram
fig,axes=plt.subplots(1,3,sharex=True,sharey=True)
ax = axes.ravel()
#m=plt.pcolormesh(X,Y,phasediagram,cmap=cmap)
m=ax[0].pcolormesh(X,Y,phasediagram,cmap=cmap)
ax[0].set_xlabel('X-ray intensity')
ax[0].set_ylabel('Neutron intensity')
ax[0].set_title('Mahlanobis distance')
ax[1].pcolormesh(X,Y,ph,cmap=cmap)
ax[1].set_xlabel('X-ray intensity')
ax[1].set_ylabel('Neutron intensity')
ax[1].set_title('Probability')
ax[2].pcolormesh(X,Y,ph-phasediagram,cmap=cmap)
ax[2].set_xlabel('X-ray intensity')
ax[2].set_ylabel('Neutron intensity')
ax[2].set_title('Difference')
#cb=fig.colorbar(m,ticks=np.arange(nphases))
m.set_clim(-0.5, nphases - 0.5)


Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [144]:
nphases

6